In [145]:
import os
import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go

In [146]:
ml = pd.read_csv('merged.csv')
pop = pd.read_excel("st-est00int-01.xls")

In [147]:
ml_2000 = ml[ml['YEAR'] == 2000]
ml_2005 = ml[ml['YEAR'] == 2005]

In [148]:
df_00 = ml_2000["STATE"].value_counts()
df_05 = ml_2005["STATE"].value_counts()

In [149]:
pop = pop.iloc[:, [0, 2, 7]]
pop = pop[8:59]
pop.columns = ['State', '2000', '2005']
pop.State = pop.State.str.strip('.')
pop['2000'] = pop['2000']/1_000_000
pop['2005'] = pop['2005']/1_000_000

In [150]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [151]:
pop['State Code'] = pop['State'].map(us_state_abbrev)
pop = pop.sort_values(by = ['State Code'])
pop = pop.set_index('State Code')


In [152]:
permil_00 = df_00/pop['2000']
permil_00 = permil.fillna(0)
permil_05 = df_05/pop['2005']
permil_05 = permil.fillna(0)

In [153]:
fig = go.Figure(data = go.Choropleth(
    locations = ml_2000['STATE'].unique(), # Spatial coordinates
    z = permil_00.astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Number of Hate Crime per Million",
))

fig.update_layout(
    title_text = 'Hate Crime Frequency by State in 2000',
    geo_scope='usa', # limit map scope to USA
)

fig.show()

In [155]:
fig = go.Figure(data = go.Choropleth(
    locations = ml_2005['STATE'].unique(), # Spatial coordinates
    z = permil_05.astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Number of Hate Crime per Million",
))

fig.update_layout(
    title_text = 'Hate Crime Frequency by State in 2005',
    geo_scope='usa', # limit map scope to USA
)

fig.show()